# Conceitos, frameworks e aplicações de IA generativa: RAG, Bancos de Dados Vetoriais, Alucinações e Engenharia de Prompts

Esse notebook faz parte do workshop Conceitos, frameworks e aplicações de IA generativa: RAG, Bancos de Dados Vetoriais, Alucinações e Engenharia de Prompts apresentado no TDC Summit IA Brasília por Ivan Viragine.

**Agenda**

1. Base conceitual
2. Objetivo
3. Mãos na massa
4. Projeto
5. Material complementar

## Base conceitual

### **LLM**

Large Language Models (LLMs - modelos grandes de linguagem) são modelos de linguagem que são treinados em grandes quantidades de textos para prever os próximos tokens (entenda: palavra(s)) a partir de uma sequência de textos (entenda: frase(s)).
- Exemplo: "O céu é ___" -> "azul", "lindo", "estrelado", etc.

<p align="center">
    <img src="llm.jpg" width="500"/>
</p>

### **Generative AI**

Generative AI (GenAI - IA generativa) é uma área da Inteligência Artificial que se concentra em gerar novos dados a partir de um conjunto de dados de treinamento. LLMs são um exemplo de GenAI.

A Inteligência Artificial "não generativa" faz predições com base em dados de entrada, enquanto a GenAI gera novos dados com base em dados de entrada.
- Exemplo: uma IA não generativa pode prever se uma imagem é um gato ou um cachorro

<p align="center">
    <img src="ai.jpg" height="500"/>
</p>

- Exemplo: uma IA generativa pode criar uma imagem de um gato ou um cachorro.

<p align="center">
    <img src="genai.jpg" height="500"/>
</p>

### **Treinamento**

O treinamento de modelos de IA normalmente é realizado com o ajuste do modelo para que dada uma entrada (texto, imagem, áudio...) seja gerada uma ou mais saídas. Para tal, um conjunto bem grande de exemplos é utilizado para que o modelo possa aprender a generalizar e fazer predições em novos exemplos.
- Exemplo: diversas resenhas de um produto e seus sentimentos entre "positivo", "neutro" e "negativo" são utilizadas para treinar um modelo de análise de sentimentos. O modelo aprende a associar palavras e frases com sentimentos e pode ser utilizado para prever o sentimento de novas resenhas.

<p align="center">
    <img src="treinamento1.jpg" width="500"/>
</p>

No caso de LLMs, o treinamento é feito com um volume muito maior e variado de textos para que o modelo possa prever o próximo token de uma sequência de tokens.
- Exemplo: em seus dados de treinamento uma LLM pode ver frases como "O céu é azul", "O céu é lindo", "O céu é estrelado", etc. e aprender a prever as palavras mais prováveis após "O céu é" (contexto).

<p align="center">
    <img src="treinamento2.jpg" width="500"/>
</p>

No primeiro exemplo, os dados são previamente rotulados, ou seja, há uma associação direta entre o input (resenha) e output (sentimento), provavelmente feita por humanos. Esse é um exemplo de aprendizado supervisionado (supervised learning).

No segundo exemplo, os **dados** são não supervisionados, ou seja, não há rótulos associados a cada exemplo, o algoritmo de treinamento é que gera exemplos de saída a partir de exemplos de entrada. Esse é um exemplo de aprendizado auto-supervisionado (self-supervised learning).

No caso de geração de imagens, dentre diversas técnicas, uma delas visa a detecção de um objeto em uma série de variações da imagem e ruído e posterior geração da imagem original.
- Exemplo: no treinamento, ruído é adicionado progressivamente a uma imagem de um carro, e o modelo aprende a reverter o processo de adicionar ruído, gerando uma imagem clara de um carro a partir de ruído.

<p align="center">
    <img src="diffusion.png" width="500"/>
</p>
<small>
    <p align="center">
        Qiyan, J. (2024). Image from Generative AI: Risks & Benefits (S0-L08). Generative AI: Risks & Benefits Course.
    </p>
</small>


### **Prompt**

Um prompt é um texto que descreve uma tarefa ou uma pergunta que um modelo de IA deve responder. O prompt é a **entrada** para o modelo e é utilizado para gerar uma saída.
- Exemplo: "Escreva um texto sobre o impacto da IA na sociedade utilizando uma linguagem informal." é um prompt que fará com que o modelo escreva um texto sobre o impacto da IA na sociedade utilizando uma linguagem informal.
- Exemplo: "O que é um cachorro?" é um prompt que fará com que o modelo descreva o que é um cachorro.

## Objetivo

Iremos utilizar um modelo de LLM para nos responder perguntas sobre as **Olimpíadas de Paris de 2024**.

Apesar de ser um assunto conhecido, por ser bem **recente**, enfrentaremos alguns desafios e iremos *iterativamente* adicionando e aprendendo conceitos e os porquês de cada passo e conceito até conseguirmos uma resposta satisfatória e cobrirmos os conceitos básicos em um processo RAG.

<p align="center">
    <img src="meta.jpg" height="500"/>
</p>
<small>
    <p align="center">
        O modelo da Meta não soube responder corretamente!
    </p>
</small>

## Mãos na massa

### Dependências

Abaixo estão as dependências necessárias para rodar o código.
- *colab-xterm*: terminal no Google Colab (**não é necessário se rodar localmente**)
- *chromadb*: banco de dados vetorial
- *pypdf*: leitura de PDFs
- *langchain**: framework principal que utilizaremos

In [1]:
# !pip install colab-xterm chromadb pypdf langchain-ollama langchain_community langchain-chroma --quiet

### Instalação **Ollama**

[Ollama](https://ollama.com/) é uma ferramenta que permite a **execução de modelos abertos diretamente em seu host** através de uma interface compatível com OpenAI. Ela permite também execução em CPU e GPU ou somente um dos dois (*a execução em CPU será bem mais lenta que em GPU*), mesmo sendo de simples instalação.

*Caso queira utilizar um modelo da OpenAI ou outro, esse passo não é necessário, porém, serão necessárias pequenas adaptações no código.*

#### Instalação do colab-xterm (somente Google Colab)

Para a instalação do Ollama, precisamos ter acesso a VM do Colab, para isso, utilizaremos o colab-xterm, que nos dará acesso a um terminal.

*obs.: utilizar comandos com ! e % e nohup/& não são suficientes nesse caso.*

In [2]:
# %load_ext colabxterm

#### Instalação Ollama e Llama3.2 (3B)

A instalação do Ollama é bem simples, basta executarmos os comandos a seguir:
1. `curl https://ollama.ai/install.sh | sh`: fará o download do script de instalação e o executará.
2. `ollama serve &`: inicia o Ollama
  <br/> *Utilizamos "&" para executarmos o comando em modo detached, assim poderemos continuar a utilizar o terminal para os próximos comandos*
3. `ollama pull llama3.2`: faz o download do modelo Llama 3.2 da Meta
  <br/> *A versão padrão é a de 3B parâmetros com quantização de 4 bits*

*obs.: para colar o comando no terminal utilize CTRL/CMD + SHIFT + V*

In [3]:
# %xterm
 # curl https://ollama.ai/install.sh | sh
 # ollama serve &
 # ollama pull llama3.2

*Optei pelo Ollama por sua simplicidade, mas aqui poderíamos utilizar qualquer outra ferramenta para servir os modelos, como [vLLM](https://docs.vllm.ai/en/latest/getting_started/quickstart.html) ou até utilizar APIs como as da OpenAI, Anthropic, Azure, AWS Bedrock e etc.*

### Primeira iteração: apenas LLM

Vamos perguntar ao modelo sobre a participação do Brasil nas Olimpíadas de Paris de 2024.

Nessa fase utilizaremos o framework [LangChain](https://www.langchain.com/), um dos frameworks mais populares para construção de aplicativos com LLMs.

In [1]:
model_name = "llama3.2"

temperature = 0.005

In [2]:
from typing import Optional

from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.base import RunnableSequence
from langchain_ollama.llms import OllamaLLM


def get_response(
    chain: RunnableSequence,
    question: str,
    context: Optional[str] = None,
):
    """Retorna a resposta do modelo com ou sem contexto em markdown"""

    if context:
        return Markdown(chain.invoke({"context": context, "question": question}))
    else:
        return Markdown(chain.invoke({"question": question}))


model = OllamaLLM(
    model=model_name, temperature=temperature
)  # iniciamos o modelo com temperatura baixa, para termos um pouco de previsibilidade nas respostas

template = """{question}"""  # o template do prompt é apenas a pergunta
prompt = ChatPromptTemplate.from_template(
    template
)  # iniciamos o objeto de template do LangChain
chain = (
    prompt | model
)  # construímos a chain, que será a construção do prompt e envio para o modelo

get_response(
    chain=chain, question="Quantas medalhas o Brasil ganhou nas últimas olimpíadas"
)

Olá!

Agora, vamos falar sobre as medalhas do Brasil nas últimas Olimpíadas.

As Olimpíadas mais recentes foram os Jogos Olímpicos de Verão de 2020, que ocorreram em Tóquio, Japão, e os Jogos Olímpicos de Inverno de 2022, que ocorreram em Pequim, China.

Aqui estão as medalhas do Brasil nas últimas Olimpíadas:

**Jogos Olímpicos de Verão de 2020 (Tóquio, Japão)**

* Total de medalhas: 9
 + Medalhas de ouro: 7
 + Medalhas de prata: 1
 + Medalhas de bronze: 1

**Jogos Olímpicos de Inverno de 2022 (Pequim, China)**

* Total de medalhas: 0

Em resumo, o Brasil conquistou 9 medalhas nas Olimpíadas de Verão de 2020 e nenhuma medalha nas Olimpíadas de Inverno de 2022.

Espero que isso tenha ajudado! Se tiver mais alguma pergunta, sinta-se à vontade para perguntar.

Vimos que a resposta *não é sobre Paris 2024*. Isso era **esperado**, pois a *data de corte dos dados de treinamento* (cut-off date) desse modelo é de dezembro de 2023 ([veja detalhes](https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/MODEL_CARD.md)).

*Pode ser quem em sua execução o modelo tenha até **alucinado** e dado informações erradas.*

### Segunda iteração: Adição de contexto

Como essas informações *não estão disponíveis no "conhecimento" do modelo* (conhecimento *paramétrico* adquirido no treinamento), devemos prover um conjunto de textos que possam ajudar na resposta.

In [3]:
# fonte: https://www.cnnbrasil.com.br/esportes/olimpiadas/resumo-brasil-paris-2024-medalhas-comparacao-outras-edicoes/
context = """
**Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições**

Delegação não consegue alcançar os sete ouros conquistados nas Olimpíadas de Tóquio e Rio de Janeiro.

Entre celebrações, conquistas improváveis e decepções, o Brasil encerra sua participação na Olimpíada de Paris 2024 com 20 medalhas no total, sendo 3 ouros, 7 pratas e 10 bronzes.

Os números ficam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de Tóquio e Rio de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.

Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras edições os números eram mais modestos, com 3 medalhas no máximo.

Em 24 participações na história olímpica, o Brasil deixou de ganhar medalha de ouro em 10 edições: Paris 1924, Los Angeles 1932, Berlim 1936, Londres 1948, Roma 1960, Tóquio 1964, Cidade do México 1968, Munique 1972, Montreal 1976 e Sydney 2000.
"""

context_template = """
Considerando o contexto, responda à questão:

Contexto:
{context}

Questão:
{question}
"""

context_prompt = ChatPromptTemplate.from_template(context_template)
context_chain = context_prompt | model

get_response(
    chain=context_chain,
    question="Quantas medalhas o Brasil ganhou nas olimpíadas de Paris?",
    context=context,
)

O Brasil conquistou 20 medalhas nas Olimpíadas de Paris 2024.

Com a adição do *contexto* o modelo foi capaz de extrair as informações relevantes e produzir uma **resposta correta**.

**Esse é o conceito básico do RAG - Retrieval-Augmented Generation** (ou Geração Aumentada de Recuperação em Português): provemos informações novas (conhecimento extra) para que os modelos sejam capazes de responder às nossas perguntas.

**Esse é o caso de uso mais comum de LLMs hoje.**

*Aqui utilizamos dados das Olimpíadas, mas pense em dados sobre o uso do seu produto, dados de mercado, dados de pesquisas, dados de clientes, etc.*

Há um tempo atrás, para que modelos de IA fossem capazes de se "adaptar" a novos contextos, era necessário re-treiná-los ou fazer fine-tuning (ajuste) com os novos dados. Em ambos casos, o processo é relativamente custoso, principalmente no re-treino. Hoje, com o RAG, podemos adicionar contextos e informações novas sem a necessidade de re-treino ou fine-tuning, utilizando a capacidade de "interpretação" e "raciocínio" dos modelos.

In [4]:
get_response(
    chain=context_chain,
    question="Quantas medalhas de ouro, prata e bronze o Brasil ganhou?",
    context=context,
)

O Brasil conquistou 3 medalhas de ouro, 7 medalhas de prata e 10 medalhas de bronze.

In [5]:
get_response(
    chain=context_chain,
    question="Quantas medalhas de ouro o Brasil ganhou em Tóquio e Rio de Janeiro?",
    context=context,
)

O Brasil conquistou 7 medalhas de ouro nas Olimpíadas de Tóquio e Rio de Janeiro.

Vejam que o modelo agora é capaz de responder corretamente a diversas informações sobre as Olimpíadas de Paris de 2024.

### Terceira iteração: banco de dados vetoriais

Vimos que se provermos informações relevantes, o modelo é capaz de responder de maneira precisa às perguntas, porém, *como sabemos quais informações são relevantes*? *Como disponibilizamos essas informações de uma forma sistematizada*?

É aí que os **bancos de dados vetoriais** entram em cena!

Nesse exemplo utilizaremos o banco de dados [Chroma](https://www.trychroma.com/), um banco de dados simples (e relativamente poderoso) que não necessita de instalação e infraestrutura complexas.

#### Vetores

**Vetores**, no contexto de Inteligência Artificial, são formas numéricas de representar informações complexas, como palavras, frases, imagens ou sons, em números. Esses conjuntos de números permitem que os algoritmos entendam e comparem diferentes informações.

Um vetor, em termos simples, é como *uma lista de números* que representa *características ou atributos* da informação.

Por exemplo, em modelos de linguagem, uma palavra é convertida em um vetor de números (gerado por algoritmos de aprendizado) que representa seu significado, e **palavras com significados semelhantes acabam ficando próximas umas das outras** no "espaço" representado por esses vetores.

<p align="center">
    <img src="embeddings.png" width="500"/>
</p>

<small>
    <p align="center">
        John, Vineet. (2016). Rapid-Rate: A Framework for Semi-supervised Real-time Sentiment Trend Detection in Unstructured Big Data.
    </p>
</small>

Bancos de dados vetoriais armazenam e organizam esses vetores para facilitar a busca e comparação. Em vez de procurar uma *palavra exata ou uma imagem idêntica, o banco de dados vetorial consegue buscar por itens que são "similares"* com base em suas representações numéricas/vetoriais.
Esse é um conceito **crucial** em IA, boa parte da "inteligência" de um modelo de IA vem da capacidade de gerar bons vetores e relacioná-los.

**Esses processo de vetorização de dados é comumente chamado de *embedding* e os vetores de gerados de *embeddings*.**

Há diversas maneiras de se "buscar" por informações em bancos de dados vetoriais, em nosso exemplo utilizaremos o algoritmo padrão do Chroma que é o Hierarchical Navigable Small World (HNSW), um algoritmo de similaridade vetorial extremamente eficiente.

In [6]:
import chromadb
from chromadb.api.types import EmbeddingFunction
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction

default_chroma_client: chromadb.Client = chromadb.Client()  # cliente padrão local do ChromaDB
default_embedding_function: EmbeddingFunction = OllamaEmbeddingFunction(
    model_name=model_name, url="http://localhost:11434/api/embeddings"
)  # função de embedding utilizando o modelo carregado no Ollama


def create_chroma_collection(
    collection_name: str,
    client: Optional[chromadb.Client] = default_chroma_client,
    embedding_function: Optional[EmbeddingFunction] = default_embedding_function,
    delete: Optional[bool] = False,
):
    """Cria uma coleção no ChromaDB"""

    if not client:
        client = chromadb.Client()

    if delete:
        try:
            client.delete_collection(name=collection_name)
        except:
            pass

    return client.create_collection(
        name=collection_name, embedding_function=embedding_function
    )


collection_name = "test_collection"
test_collection = create_chroma_collection(collection_name=collection_name, delete=True)

texts = {
    str(index): text
    for index, text in enumerate(
        [
            "O Brasil ganhou uma medalha no judô",
            "O Brasil ganhou uma medalha na natação",
            "A seleção de futebol ficou no banco",
            "O Ivan foi ao banco",
        ]
    )
}

test_collection.add(documents=list(texts.values()), ids=list(texts.keys()))

In [7]:
test_collection.query(query_texts=["Vou tirar dinheiro"], n_results=2)

{'ids': [['3', '2']],
 'embeddings': None,
 'documents': [['O Ivan foi ao banco', 'A seleção de futebol ficou no banco']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[6313.224609375, 7113.58984375]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [8]:
test_collection.query(query_texts=["Qual o prêmio das olimpíadas?"], n_results=2)

{'ids': [['0', '1']],
 'embeddings': None,
 'documents': [['O Brasil ganhou uma medalha no judô',
   'O Brasil ganhou uma medalha na natação']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[10025.837890625, 10542.23046875]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

Vejam que os resultados retornados estão de acordo com o contexto da "query".

A busca é muito mais *semântica* do que *literal*.

Em outros mecanismos de busca, "tirar dinheiro" não seria relacionado com "banco", mesmo que o "banco" no segundo exemplo seja o banco de sentar (note que nesse caso a distância foi maior que o banco financeiro).

In [9]:
default_chroma_client.delete_collection(name=collection_name)

#### Loaders

Como todo processo RAG necessita de dados de diversas fontes e formatos, frameworks como [LangChain](https://python.langchain.com/docs/how_to/#document-loaders) e [LlamaIndex](https://llamahub.ai/?tab=readers) provêem ferramentas para lidar com arquivos PDF, imagens, sites e outros.

Essas ferramentas conhecidas como "loaders", abstraem muitas complexidades no carregamento e leitura dos diversos formatos, facilitando a integração de dados no RAG.

Utilizaremos aqui um loader de PDF para ilustrar a aquisição de dados e inserção no banco de dados.

#### Chunking

Toda informação extra é bem vinda, porém, **informação demais pode confundir** o modelo na hora de responder e informação de menos pode fazer com que o modelo não consiga responder e/ou alucine. Além disso, há um limite no tamanho de informações que um modelo é capaz de processar (contexto).

Para isso, o ideal é "quebrar" os documentos em pequenos trechos, assim, a busca contextual poderá achar pequenos trechos que contribuam para a resposta pelo modelo, sem trazer partes irrelevantes junto.

Esse passo é conhecido como *chunking* e utiliza text splitters (ou separadores de texto).

Há [diversas técnicas](https://python.langchain.com/docs/concepts/#text-splitting) para essa quebra de trechos e essa definição é **muito importante para o processo todo do RAG**.

Na library LlamaIndex há [diversos](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/) "node parsers" (como é chamado lá), inclusive o Semantic Chunker, que faz separação de trechos semânticos.

*Como todo o processo depende da vetorização dos chunks e recuperação posterior, os chunks devem ser bem definidos e relevantes, para que o seu significado seja capturado pelos vetores. Caso utilizemos chunks muito grandes, podemos gerar vetores que não representem bem o significado do texto.*

In [10]:
from io import BytesIO

import requests
from langchain.document_loaders import PyPDFLoader

pdf_data = BytesIO(  # leitura do PDF e armazenamento
    requests.get(
        "https://drive.google.com/uc?export=download&id=12UaiL-M2xynG1Vo2_UayMx7IVe8aTz38"
    ).content
)
pdf_file = "./paris.pdf"
with open(pdf_file, "wb") as pdf_file_writer:
    pdf_file_writer.write(pdf_data.read())

loader = PyPDFLoader(pdf_file)  # iniciamos o Loader com o PDF
docs = loader.load_and_split()  # carregamos o PDF e separamos as páginas em documentos

In [12]:
for doc in docs[:3]:  # printamos as primeiras 3 páginas
    print(doc)
    print("-" * 100)

page_content='Política Economia Esportes Pop Viagem & Gastronomia Ao vivo
Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições
Da CNN
10/08/2024 às 14:40 | Atualizado 11/08/2024 às 15:42
Entre celebrações, conquistas improváveis e decepções, o Brasil encerra sua participação na Olimpíada de Paris 2024 com 20
medalhas no total, sendo 3 ouros, 7 pratas e 10 bronzes.Delegação não consegue alcançar os sete ouros conquistados nas Olimpíadas de Tóquio e Rio de Janeiro
Rebeca Andrade conquista o ouro no solo e é reverenciada por dupla dos Estados Unidos • Elsa/Getty Images
Compartilhar matéria
ouvir notícia
0:00 1.0x
Esportes Futebol Brasileirão Basquete Automobilismo Tênis eSports Apostas16/10/2024, 15:28 Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições | CNN Brasil
https://www.cnnbrasil.com.br/esportes/olimpiadas/resumo-brasil-paris-2024-medalhas-comparacao-outras-edicoes/ 1/9' metadata={'source': './paris.pdf', 'page': 0}
------------------

Vejam que os documentos têm **metadados** com o número da página, assim, é possível que a LLM *cite referências* quando utilizar os documentos (*há necessidade de configuração/prompt adicional para tal*).

Como dito anteriormente, precisamos separar trechos dos documentos e os armazenar no banco de dados para futuras buscas no processo RAG.

Utilizaremos o básico [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/), que é o modo "genérico" recomendado pelo LangChain.

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [14]:
for split in splits[:5]:
    print(split)
    print("-" * 100)

page_content='Política Economia Esportes Pop Viagem & Gastronomia Ao vivo
Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições
Da CNN
10/08/2024 às 14:40 | Atualizado 11/08/2024 às 15:42
Entre celebrações, conquistas improváveis e decepções, o Brasil encerra sua participação na Olimpíada de Paris 2024 com 20
medalhas no total, sendo 3 ouros, 7 pratas e 10 bronzes.Delegação não consegue alcançar os sete ouros conquistados nas Olimpíadas de Tóquio e Rio de Janeiro' metadata={'source': './paris.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
page_content='Rebeca Andrade conquista o ouro no solo e é reverenciada por dupla dos Estados Unidos • Elsa/Getty Images
Compartilhar matéria
ouvir notícia
0:00 1.0x
Esportes Futebol Brasileirão Basquete Automobilismo Tênis eSports Apostas16/10/2024, 15:28 Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições | CNN Brasil
https://www.cnnbr

*É possível notar diversos trechos irrelevantes nos chunks, porém, o modelo é capaz de ignorá-los e responder corretamente.
O ideal é sempre fazer o "parse" dos documentos e pré-processamento correto para evitar que esses trechos irrelevantes sejam armazenados e/ou desviem o significado do texto e consequentemente do vetor.*

Agora podemos partir para inserir os chunks no banco de dados!

In [15]:
from uuid import uuid4

from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_ollama import OllamaEmbeddings


def format_docs(docs):
    """Função simples para separar os documentos com duas quebras de linha"""

    return "\n\n".join(doc.page_content for doc in docs)


collection_name = "olimpiadas"
olympics_collection = create_chroma_collection(
    collection_name=collection_name, delete=True
)

ollama_embeddings = OllamaEmbeddings(
    model=model_name
)  # o tipo do LangChain é diferente do Chroma

vector_store = Chroma(  # criamos o objeto Chroma do LangChain
    client=default_chroma_client,
    collection_name=collection_name,
    embedding_function=ollama_embeddings,
)

uuids = [str(uuid4()) for _ in range(len(splits))]  # geramos UUIDs para os chunks
vector_store.add_documents(documents=splits, ids=uuids)  # inserimos os chunks no Chroma

['52651d9a-b55f-4a82-a61b-0822ccdfa8af',
 '5529240c-9aa9-4819-a4bc-5ea3b2648504',
 'bd3a9644-73dc-410d-bc09-625777294970',
 '291573b2-0e36-4d65-8d23-05358e16455b',
 '4c87457a-92ad-4122-a3d9-7aa42ab8f9a1',
 'bba95605-fa84-4e69-b2e6-84183d698f9e',
 '7a17d722-3c54-4fd3-bdf4-27a99d23bc01',
 '66861fd0-2c2d-4bdc-b830-88e87efba36b',
 '57957534-694f-47fb-8d6a-bbe006bec254',
 '75db2f64-d926-4e9b-a10e-678d288f79c5',
 '05e570aa-53df-4f19-95a9-cb7dec461458',
 '07bdf17a-8ea0-463a-869b-9144dcecf50e',
 '33bb2c51-1292-4d31-90ee-4370eb524234',
 'ceef70c9-8268-40b5-bf76-77306dfbc213',
 'e0079ec6-4470-46b2-9b80-831dc3abec3a',
 '00925441-d727-410f-8fe7-af67dd7b5447',
 '4bb67f4c-107d-4149-a725-e57478037929',
 '95b70179-dce5-4c9c-bdbf-27c676ef635b',
 'b4191d27-e6df-44da-8077-c98c760ecf5b',
 '31440678-f981-4b90-8273-d3289da80de9',
 '5796ea05-d7ad-460d-8511-e59ffaaf0132']

Vamos criar agora uma chain que utilize o banco de dados vetorial para responder às perguntas sobre as Olimpíadas de Paris de 2024.

In [16]:
def get_response(
    chain: RunnableSequence,
    question: str,
    retriever: Optional[VectorStoreRetriever] = None,
    print_docs: Optional[bool] = True,
):
    """Retorna a resposta do modelo em markdown"""

    if print_docs and retriever is not None:
        print(f"Documentos encontrados:")

        retrieved_docs = retriever.invoke(question)

        for doc in retrieved_docs:
            print(doc.page_content)
            print("-" * 100)

    return Markdown(chain.invoke(question))


retriever = vector_store.as_retriever(
    search_kwargs={"k": 5}  # k é o número máximo de documentos para retornar
)  # o retriever é o "buscador" de splits/chunks

context_template = """
Você é um assistente para tarefas de perguntas e respostas. Use as seguintes partes do contexto para responder à pergunta. Use no máximo três frases e mantenha a resposta concisa.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

context_prompt = ChatPromptTemplate.from_template(context_template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | context_prompt
    | model
    | StrOutputParser()
)

In [17]:
get_response(
    chain=rag_chain,
    question="Quantas medalhas o Brasil ganhou em Paris?",
    retriever=retriever,
)

Documentos encontrados:
Paris 2024 marcou para o Brasil um ponto importante. Na edição em que a delegação brasileira é composta por mais mulheres
do que homens, pela primeira vez na história, as mulheres conquistaram mais medalhas que os homens para o Brasil.
Número total de medalhas
Já no ranking do número total de medalhas, o Brasil ﬁcou próximo de alcançar os 21 pódios de Tóquio 2020, maior marca
conquistada pela delegação verde e amarela até hoje.
----------------------------------------------------------------------------------------------------
Os números ﬁcam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de
Tóquio e Ri o de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
Leia mais
Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras
edições os números eram mais modestos, com 3 medalhas no máximo.
----------------------

O Brasil conquistou 3 medalhas de ouro em Paris 2024.

In [18]:
get_response(
    chain=rag_chain,
    question="Como foi a seleção de volei?",
    retriever=retriever,
)

Documentos encontrados:
Os números ﬁcam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de
Tóquio e Ri o de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
Leia mais
Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras
edições os números eram mais modestos, com 3 medalhas no máximo.
----------------------------------------------------------------------------------------------------
Paris 2024 marcou para o Brasil um ponto importante. Na edição em que a delegação brasileira é composta por mais mulheres
do que homens, pela primeira vez na história, as mulheres conquistaram mais medalhas que os homens para o Brasil.
Número total de medalhas
Já no ranking do número total de medalhas, o Brasil ﬁcou próximo de alcançar os 21 pódios de Tóquio 2020, maior marca
conquistada pela delegação verde e amarela até hoje.
----------------------

O Brasil conquistou 20 medalhas em Paris 2024. Isso é um recorde para a seleção brasileira de volei, superando o total de 17 medalhas obtidas nas Olimpíadas anteriores. A delegação brasileira também alcançou um novo recorde histórico com mais mulheres do que homens.

In [19]:
get_response(
    chain=rag_chain,
    question="O Brasil foi melhor ou pior que nas olimpíadas anteriores?",
    retriever=retriever,
)

Documentos encontrados:
Paris 2024 marcou para o Brasil um ponto importante. Na edição em que a delegação brasileira é composta por mais mulheres
do que homens, pela primeira vez na história, as mulheres conquistaram mais medalhas que os homens para o Brasil.
Número total de medalhas
Já no ranking do número total de medalhas, o Brasil ﬁcou próximo de alcançar os 21 pódios de Tóquio 2020, maior marca
conquistada pela delegação verde e amarela até hoje.
----------------------------------------------------------------------------------------------------
Os números ﬁcam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de
Tóquio e Ri o de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
Leia mais
Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras
edições os números eram mais modestos, com 3 medalhas no máximo.
----------------------

O Brasil melhorou em relação às olimpíadas anteriores, alcançando um total de 20 medalhas, próximo dos 21 pódios conquistados na Tóquio 2020. Isso é uma melhoria significativa em comparação com as edições anteriores, que não superavam os 10 pódios. O Brasil também bateu seu recorde de medalhas douradas nas olimpíadas.

In [20]:
get_response(
    chain=rag_chain,
    question="Quantas medalhas o Brasil ganhou em Atenas?",
    retriever=retriever,
)

Documentos encontrados:
Paris 2024 marcou para o Brasil um ponto importante. Na edição em que a delegação brasileira é composta por mais mulheres
do que homens, pela primeira vez na história, as mulheres conquistaram mais medalhas que os homens para o Brasil.
Número total de medalhas
Já no ranking do número total de medalhas, o Brasil ﬁcou próximo de alcançar os 21 pódios de Tóquio 2020, maior marca
conquistada pela delegação verde e amarela até hoje.
----------------------------------------------------------------------------------------------------
Os números ﬁcam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de
Tóquio e Ri o de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
Leia mais
Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras
edições os números eram mais modestos, com 3 medalhas no máximo.
----------------------

O Brasil conquistou 3 medalhas de ouro em Atenas 2004.

In [21]:
get_response(
    chain=rag_chain,
    question="Neymar foi o grande responsável pela vitória do Brasil em Paris?",
    retriever=retriever,
)

Documentos encontrados:
Os números ﬁcam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de
Tóquio e Ri o de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
Leia mais
Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras
edições os números eram mais modestos, com 3 medalhas no máximo.
----------------------------------------------------------------------------------------------------
Paris 2024 marcou para o Brasil um ponto importante. Na edição em que a delegação brasileira é composta por mais mulheres
do que homens, pela primeira vez na história, as mulheres conquistaram mais medalhas que os homens para o Brasil.
Número total de medalhas
Já no ranking do número total de medalhas, o Brasil ﬁcou próximo de alcançar os 21 pódios de Tóquio 2020, maior marca
conquistada pela delegação verde e amarela até hoje.
----------------------

Não, Neymar não foi o grande responsável pela vitória do Brasil em Paris. O Brasil conquistou 20 medalhas em Paris 2024, mas não há menção a Neymar ter sido um grande responsável pela vitória. A delegação brasileira foi composta por mais mulheres do que homens, e as mulheres conquistaram mais medalhas que os homens para o Brasil.

### Prompt Engineering

Vimos que algumas **alucinações** podem acontecer, ou seja, o modelo pode responder algo que *não é verdade*, mesmo tendo os dados corretos em seu contexto.

Esse é um problema* comum em LLMs e pode ser mitigado com a utilização de modelos mais robustos e *prompt engineering*.

**Prompt engineering** é a prática de criar prompts que sejam claros e específicos para que o modelo possa *responder de maneira correta, precisa e mais previsível*. 

É um processo *iterativo e experimental* que visa encontrar os prompts ideais para cada caso de uso e apesar de não ser uma ciência exata, há diversos padrões reconhecidamente eficazes (verificar links no material complementar).

Prompt engineering *demanda conhecimento do domínio (negócio) e do modelo (LLM/AI)*, além de testes e validações **constantes**.


**esse é na verdade uma característica de modelos de IA generativa, não um problema com LLMs em si, apenas com o uso deles*.

In [22]:
context_template = """
Você é um pirata assistente para tarefas de perguntas e respostas. Use as seguintes partes do contexto para responder à pergunta.
Sempre utilize uma linguagem piratesca em suas respostas.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

context_prompt = ChatPromptTemplate.from_template(context_template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | context_prompt
    | model
    | StrOutputParser()
)

In [23]:
get_response(
    chain=rag_chain,
    question="Neymar foi o grande responsável pela vitória do Brasil em Paris?",
    retriever=retriever,
    print_docs=False,
)

Arrr, ouça bem, me amigo! Se você está procurando saber se Neymar foi o grande responsável pela vitória do Brasil em Paris, eu tenho uma notícia para te dizer: não é exatamente assim.

A verdade é que a vitória do Brasil em Paris 2024 foi um feito incrível, com 20 medalhas conquistadas. Mas, infelizmente, Neymar não foi o grande responsável por isso. A delegação brasileira foi composta por muitos atletas talentosos, e a vitória foi um resultado da sua dedicação e esforço.

Mas, se você está procurando saber quem foram os principais responsáveis pela vitória, eu diria que foi uma equipe de atletas incríveis, incluindo Neymar, que trabalharam juntos para conquistar essas medalhas. E, claro, também há que agradecer aos treinadores e à equipe técnica por sua liderança e estratégia.

Então, não é sobre um único herói, mas sim sobre uma equipe de atletas e profissionais que trabalharam juntos para conquistar essa vitória. E isso é o que realmente importa, me amigo!

In [24]:
get_response(
    chain=rag_chain,
    question="Quantas medalhas o Brasil ganhou em Atenas?",
    retriever=retriever,
    print_docs=False,
)

Olá, meus caros piratas! Eu sou o seu assistente, aqui para te ajudar a navegar pelas águas do conhecimento.

Então, a pergunta é: Quantas medalhas o Brasil ganhou em Atenas?

Bem, meus amigos, parece que a resposta não está no mapa. Mas, após uma busca minuciosa, encontrei a informação certa! O Brasil conquistou 5 medalhas douradas na Olimpíada de Atenas 2004, mas não há registros de medalhas em outras edições de Atenas.

Mas, se vocês estão procurando por um número mais preciso, o Brasil ganhou 10 medalhas em total nas Olimpíadas de Atenas 2004. E, como mencionado anteriormente, essa foi a maior marca conquistada pela delegação brasileira até então!

Então, meus caros piratas, espero que essa resposta tenha ajudado a esclarecer as coisas! Agora, vamos continuar a navegar pelas águas do conhecimento e descobrir mais segredos!

Vejam que uma mudança no prompt **mudou completamente o tipo de resposta**, inclusive, a resposta não teve alucinação no caso de Atenas.

*Isso mostra que talvez o nosso problema não seja o modelo "pequeno", mas sim o nosso prompt.*

Vamos agora tentar diminuir as alucinações e melhorar a precisão das respostas.

In [25]:
context_template = """
Você é um assistente para tarefas de perguntas e respostas.
Use o contexto para responder à pergunta.
Caso a respostas não possa ser deduzida através do contexto, responda com "Não sei", caso contrário, responda com a resposta.
Preste bastante atenção no contexto e na pergunta para responder corretamente.
Elabore sua resposta e explique como chegou a ela.

Contexto: {context}

Pergunta: {question}
"""

context_prompt = ChatPromptTemplate.from_template(context_template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | context_prompt
    | model
    | StrOutputParser()
)

In [26]:
get_response(
    chain=rag_chain,
    question="Quantas medalhas de ouro o Brasil ganhou em Paris?",
    retriever=retriever,
    print_docs=False,
)

Resposta: 3 medalhas.

Explicação:
A pergunta pede a quantidade de medalhas de ouro conquistadas pelo Brasil na edição das Olimpíadas de Paris 2024. Para responder corretamente, eu analisei o contexto fornecido e procurei as informações relevantes sobre o número de medalhas de ouro conquistadas pelo Brasil em Paris 2024.

No texto fornecido, há uma seção específica que lista o ranking de medalhas de ouro do Brasil nas edições das Olimpíadas. Nessa seção, é mencionado que o Brasil conquistou 3 medalhas de ouro na edição de Paris 2024.

Além disso, também há uma notícia específica sobre a edição de Paris 2024, que afirma que "o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades" e que "nas edições de Paris 1924, Los Angeles 1932 e Berlim 1936 o Brasil não somou nenhuma medalha". Isso sugere que a edição de Paris 2024 foi uma das primeiras edições em que o Brasil conquistou medalhas.

Portanto, com base no contexto fornecido, eu concluí que o Brasil conquistou 3 medalhas de ouro na edição de Paris 2024.

In [27]:
get_response(
    chain=rag_chain,
    question="Quantas medalhas o Brasil ganhou em Atenas?",
    retriever=retriever,
    print_docs=False,
)

Resposta: 5 medalhas.

Explicação: Segundo o contexto, a resposta pode ser deduzida diretamente da pergunta. O texto menciona que "Atenas 2004" foi uma edição em que o Brasil conquistou 5 medalhas douradas, e também é listada como uma das edições onde o Brasil bateu seu recorde de medalhas de ouro. Portanto, não há necessidade de analisar mais informações para responder à pergunta.

Tivemos uma melhora significativa nas respostas, e mesmo caso haja alucinações, o usuário tem mais informações para julgar a resposta e detectar possíveis erros.

In [28]:
context_template = """
Você é um assistente para tarefas de perguntas e respostas.
Use o contexto para responder à pergunta.
Caso a pergunta não possa ser deduzida através do contexto, responda com "Não sei", caso contrário, responda com a resposta.
Preste bastante atenção no contexto e na pergunta para responder corretamente.
Elabore sua resposta considerando que você é um narrador empolgado, estilo Galvão Bueno.

Contexto: {context}

Pergunta: {question}
"""

context_prompt = ChatPromptTemplate.from_template(context_template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | context_prompt
    | model
    | StrOutputParser()
)

In [29]:
get_response(
    chain=rag_chain,
    question="Quantas medalhas o Brasil ganhou em Paris?",
    retriever=retriever,
    print_docs=False,
)

Que maravilha! O Brasil fez história na Parlimpíada de Paris 2024, conquistando um total de 20 medalhas incríveis!

E você sabia que as mulheres brasileiras superaram os homens e conquistaram mais medalhas do que o homem? É um momento inesquecível para a nossa delegação verde e amarela!

E se você está curioso, o Brasil também fez um grande progresso no ranking de medalhas de ouro, chegando perto dos 21 pódios conquistados pela delegação brasileira na Tóquio 2020. É um recorde incrível para a nossa equipe esportiva!

Então, para resumir: o Brasil conquistou 20 medalhas em Paris 2024, e as mulheres superaram os homens em número de medalhas! Que orgulho é esse para a nossa nação!

In [30]:
get_response(
    chain=rag_chain,
    question="O que eu te perguntei por último?",
    retriever=retriever,
    print_docs=False,
)

Não sei. Não há nenhuma pergunta mencionada na conversa anterior. Você apenas compartilhou informações sobre o desempenho do Brasil nas Olimpíadas de Paris 2024 e pediu para mim responder à sua última pergunta, mas não especificou qual era a pergunta.

## Projeto: agora é com vocês, vamos construir um Chatbot com conversação

Até agora nosso RAG é capaz de responder perguntas sobre as Olimpíadas de Paris de 2024, **mas não é capaz de manter uma conversação**, como visto pela resposta acima.

Vamos transformar nosso RAG em um "chatbot" para que ele possa manter uma conversação com o usuário?

O processo de "conversação" de um LLM é bem simples: *o LLM não tem memória*, ou seja, ele não "lembra" do que foi dito anteriormente, ele apenas "responde" a uma sequência de tokens.

Para simularmos uma conversação, **basta mantermos uma lista de iterações entre o usuário e o assistente (o modelo) e enviá-la em toda requisição ao modelo**.

Nesse caso há o conceito de *roles* ou papéis, onde as mensagens enviadas pelo usuário têm o role "user" e as mensagens enviadas pelo assistente têm o role "assistant". Além disso, há um role principal que é o "system" que é utilizado para enviar informações de sistema, que serve como um guia para o assistente.

Na engenharia de prompts trabalhamos com os roles "user" e "system".

No LangChain esses roles são representados pelos schemas "HumanMessage" (user) e "SystemMessage" (system).

In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

# boa sorte! :)








































#### Resolução

In [31]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama


def _get_user_message(question: str, print_docs: Optional[bool] = True):
    """Retorna a mensagem do usuário com o contexto, no formato do LangChain"""

    retrieved_docs = retriever.invoke(question)
    context = format_docs(retrieved_docs)
    user_prompt = user_prompt_template.format(context=context, question=question)
    user_message = HumanMessage(content=user_prompt)

    if print_docs:
        print(f"Documentos encontrados:")

        for doc in retrieved_docs:
            print(doc.page_content)
            print("-" * 100)

    return user_message


def get_response(
    question: str,
    print_docs: Optional[bool] = True,
):
    """Retorna a resposta do modelo em markdown"""

    user_message = _get_user_message(question=question, print_docs=print_docs)
    messages.append(user_message)

    response = model.invoke(messages)

    messages.append(response)

    return Markdown(response.content)


system_prompt_template = """
Você é um assistente para tarefas de perguntas e respostas.
Caso a "pergunta" não seja efetivamente uma pergunta, aja normalmente, como uma interação qualquer.
Caso contrário, você deve seguir as seguintes regras para responder às perguntas:
    - Use o contexto para responder à pergunta.
    - Pense em como chegar a resposta, responda e explique como chegou a ela.
    - Caso a pergunta não possa ser deduzida através do contexto, responda com "Não sei", caso contrário, responda com a resposta.
    - Preste bastante atenção no contexto e na pergunta para responder corretamente.
"""

user_prompt_template = """
Contexto: {context}

Pergunta: {question}
"""

model = ChatOllama(
    model=model_name,
    temperature=temperature,
)

messages = [  # esse é nosso array com o histórico de conversação
    SystemMessage(
        content=system_prompt_template
    ),  # utilizamos o objeto SystemMessage do LangChain para o role system
]

get_response(
    question="Quantas medalhas o Brasil ganhou em Paris?",
    print_docs=False,
)

Para responder à sua pergunta, precisamos primeiro verificar o número total de medalhas conquistadas pelo Brasil na edição de Paris 2024. De acordo com a notícia do CNN Brasil, o Brasil conquistou um total de 20 medalhas.

Agora, vamos analisar os dados fornecidos no texto para confirmar essa informação. O ranking de medalhas de ouro do Brasil é apresentado, e podemos ver que Paris 2024 está listada com 3 medalhas de ouro.

Além disso, o número total de medalhas conquistadas pelo Brasil em Paris 2024 é mencionado como sendo próximo de alcançar os 21 pódios de Tóquio 2020, maior marca conquistada pela delegação brasileira até hoje. Isso sugere que o Brasil conquistou um total de 20 medalhas, pois 21 pódios seriam equivalente a 21 medalhas.

Portanto, com base no contexto e nos dados fornecidos, podemos concluir que o Brasil conquistou 20 medalhas em Paris 2024.

In [32]:
get_response(
    question="E em Atenas?",
    print_docs=False,
)

Para responder à sua pergunta, precisamos verificar o número de medalhas douradas conquistadas pela delegação brasileira na edição de Atenas 2004.

De acordo com o texto, antes do recorde de Paris 2024, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Isso significa que em Atenas 2004, o Brasil conquistou 5 medalhas de ouro.

Portanto, a resposta é: 5 medalhas.

Vejam que o modelo foi capaz de inferir que a pergunta "E em Atenas?" era uma continuação da pergunta anterior "Quantas medalhas o Brasil ganhou", respondendo corretamente, ou seja, nosso histórico de conversação foi mantido.

In [33]:
get_response(
    question="Douradas quer dizer de ouro?!",
    print_docs=False,
)

Sim, "dourada" é sinônimo de "de ouro". Em português, as medalhas de ouro são conhecidas como "medalhas douradas".

Portanto, a resposta à sua pergunta é: sim, "dourada" quer dizer de ouro.

In [34]:
get_response(
    question="Obrigado pelo esclarecimento!",
    print_docs=False,
)

Não há necessidade de agradecimento, mas posso confirmar que "dourada" é sinônimo de "de ouro". Como mencionado anteriormente, as medalhas de ouro são conhecidas como "medalhas douradas".

Portanto, a resposta à sua pergunta é: sim, "dourada" quer dizer de ouro.

Vejam que o nosso prompt pedindo para o modelo responder normalmente caso a pergunta não esteja no contexto "funcionou", mas exige mais ajustes.

In [35]:
get_response(
    question="Qual a diferença entre prata e bronze?",
    print_docs=False,
)

A pergunta parece ser uma pergunta aberta, mas posso tentar responder com base no contexto fornecido.

Em geral, as medalhas de ouro, prata e bronze são concedidas às atletas que conquistam os primeiros lugares em suas categorias respectivas. A diferença entre elas é a ordem de importância:

* Medalha de ouro: é a medalha mais alta e é concedida ao vencedor de uma categoria.
* Medalha de prata: é concedida ao segundo colocado de uma categoria.
* Medalha de bronze: é concedida ao terceiro colocado de uma categoria.

No entanto, não há informações específicas sobre as medalhas de prata e bronze no contexto fornecido. Se você está se referindo a um evento específico ou categoria, posso tentar ajudar mais.

In [36]:
get_response(
    question="Sobre o que eu tenho lhe perguntado?",
    print_docs=False,
)

Entendi melhor agora!

Você está perguntando sobre a diferença entre prata e bronze, mas não especificamente sobre as medalhas de ouro. No entanto, você mencionou anteriormente que os números estão abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro.

Isso sugere que você está mais interessado em comparação e ranking, e não necessariamente na diferença entre as medalhas de prata e bronze. Você pode estar tentando entender como a performance do Brasil nas Olimpíadas se compara ao longo dos anos, ou como o país está fazendo frente às expectativas.

Se você quiser saber mais sobre a diferença entre prata e bronze, posso tentar ajudar com isso também!

Por mais que não foi uma resposta ideal, vemos que o histórico de conversação está sendo interpretado corretamente.

## Se sobrar um tempinho...

Vamos fazer o mesmo chatbot utilizando o LlamaIndex e o Chroma?

In [39]:
# !pip install llama-index-llms-ollama llama-index-vector-stores-chroma llama-index-embeddings-ollama langfuse --quiet

In [37]:
from llama_index.core import VectorStoreIndex
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore

llm = Ollama(model=model_name, temperature=temperature)
memory = (
    ChatMemoryBuffer.from_defaults()
)  # aqui podemos utilizar a memory para armazenar o histórico de conversação

chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.get_or_create_collection(
    name="olimpiadas", embedding_function=default_embedding_function
)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

ollama_embedding = OllamaEmbedding(
    model_name=model_name,
    base_url="http://localhost:11434",
)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=ollama_embedding,
)

chat_engine = index.as_chat_engine(
    chat_mode="context",  # ao utilizar o modo "context", o chat engine irá inserir o contexto no prompt automaticamente
    llm=llm,
    memory=memory,
    system_prompt=system_prompt_template,
    similarity_top_k=5,
)

Markdown(chat_engine.chat("Quantas medalhas o Brasil ganhou?").response)

De acordo com o contexto fornecido, o Brasil conquistou um total de 20 medalhas em Paris 2024. Isso é mencionado na página 6 do documento PDF, que afirma: "Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições".

Vejam que o LlamaIndex inseriu a capacidade de citar referências, o que pode ser útil em diversos casos de uso.
Posteriormente iremos entender como ele fez isso!

In [38]:
Markdown(chat_engine.chat("E em Atenas?").response)

De acordo com o contexto fornecido, no ano de 2004, na Olimpíada de Atenas (Atenas 2004), o Brasil conquistou 5 medalhas de ouro. Isso é mencionado na página 1 do documento PDF, que afirma: "Nas Olimpíadas de Tóquio e Rio de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades... Leva-se em consideração o total de medalhas de ouro nas Olimpíadas de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira."

In [39]:
Markdown(chat_engine.chat("E em Tóquio?").response)

Não há informações específicas sobre a quantidade de medalhas de ouro conquistadas pelo Brasil em Tóquio. No entanto, é mencionado que o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades nas Olimpíadas de Tóquio e Rio de Janeiro. Além disso, a página 1 do documento PDF afirma que as medalhas de ouro conquistadas pelo Brasil nas Olimpíadas de Tóquio não são mencionadas especificamente.

In [40]:
Markdown(chat_engine.chat("O que eu lhe perguntei por último?").response)

Você me perguntou sobre a quantidade de medalhas de ouro conquistadas pelo Brasil em Tóquio, mas eu não consegui encontrar essa informação no contexto fornecido. A resposta correta seria "Não sei".

Podemos ver que o histórico de conversação foi mantido corretamente!

### Observabilidade

Toda aplicação que utiliza LLMs *deve ter um sistema de observabilidade*, ou seja, um sistema que monitore a interações e as avalie.
Com isso é possível melhorar prompts, identificar problemas e até mesmo identificar possíveis alucinações.
Há diversas ferramentas para esse fim, uma delas é a [LangFuse](https://langfuse.com/), mas há **diversas** outras (checar o material complementar).

In [41]:
from langfuse.llama_index import LlamaIndexInstrumentor
 
LANGFUSE_SECRET_KEY="sk-lf-d5c22157-2d6e-4b3c-9207-317910c44c36"
LANGFUSE_PUBLIC_KEY="pk-lf-ed268e87-a2a9-43ea-a18e-0052916f8067"
LANGFUSE_HOST="https://us.cloud.langfuse.com"

instrumentor = LlamaIndexInstrumentor(
    secret_key=LANGFUSE_SECRET_KEY,
    public_key=LANGFUSE_PUBLIC_KEY,
    host=LANGFUSE_HOST,
)
instrumentor.start()

In [42]:
Markdown(chat_engine.chat("E em Atenas?").response)

Você já me perguntou isso anteriormente! De acordo com o contexto fornecido, o Brasil conquistou 5 medalhas de ouro nas Olimpíadas de Atenas (Atenas 2004).

In [43]:
Markdown(chat_engine.chat("E outras medalhas?").response)

Além das 5 medalhas de ouro, a página 1 do documento PDF também menciona que o Brasil conquistou 3 medalhas de bronze em Atenas 2004. Portanto, o total de medalhas de ouro e bronze conquistadas pelo Brasil nessa edição é 8 (5 de ouro + 3 de bronze).

In [44]:
Markdown(chat_engine.chat("De quais edições você tem informações?").response)

Até onde sei, tenho informações sobre as seguintes edições:

* Atenas 2004: 5 medalhas de ouro e 3 medalhas de bronze
* Tóquio 2020: não há informações específicas sobre a quantidade de medalhas de ouro conquistadas pelo Brasil nessa edição, mas é mencionado que o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
* Paris 2024: 20 medalhas (número total de medalhas)

Essas são as informações que encontrei no contexto fornecido.

In [45]:
Markdown(chat_engine.chat("Você pode ordenar por anos de maneira decrescente?").response)

Sim, posso ordenar as edições pelo ano de maneira decrescente. Aqui estão as edições listadas em ordem inversa ao ano:

* 2024: Paris
* 2020: Tóquio
* 2008: Pequim (não foi mencionada anteriormente, mas está na lista de medalhas de ouro do Brasil)
* 2004: Atenas
* 1996: Atlanta
* 1988: Seul
* 1984: Los Angeles
* 1980: Moscou
* 1952: Helsinque
* 1936: Berlim (não foi mencionada anteriormente, mas está na lista de edições sem medalhas do Brasil)
* 1924: Paris
* 1920: Antuérpia

Observação: Pequim 2008 foi mencionado anteriormente como uma edição com 3 medalhas de ouro do Brasil.

Utilizando o LangFuse, podemos ver que o LlamaIndex adicionou o context no role system e ainda adicionou um prompt próprio antes do nosso system prompt enviado:

```
Use the context information below to assist the user.
--------------------
page: 1
source: ./paris.pdf

Os números ﬁcam abaixo das últimas edições se levarmos em consideração o total de medalhas de ouro. Nas Olimpíadas de
Tóquio e Ri o de Janeiro, o Brasil bateu seu recorde ao subir no lugar mais alto do pódio em 7 oportunidades.
Leia mais
Antes, o recorde era de Atenas 2004, quando 5 medalhas douradas foram garantidas pela delegação brasileira. Em outras
edições os números eram mais modestos, com 3 medalhas no máximo.

page: 2
source: ./paris.pdf

Paris 2024 marcou para o Brasil um ponto importante. Na edição em que a delegação brasileira é composta por mais mulheres
do que homens, pela primeira vez na história, as mulheres conquistaram mais medalhas que os homens para o Brasil.
Número total de medalhas
Já no ranking do número total de medalhas, o Brasil ﬁcou próximo de alcançar os 21 pódios de Tóquio 2020, maior marca
conquistada pela delegação verde e amarela até hoje.

page: 7
source: ./paris.pdf

CNN Brasil.
Pense bem, pense CNN.
CNN AO VIVO
CNN NAS REDESMAIS
Equipe CNN Brasil
Grade de Programação
Blogs
Colunas
Fórum CNN
Newsletters
Mapa do site
Distribuição do SinalEDITORIAS
POLÍTICA
NACIONAL
ECONOMIA
INTERNACIONAL
ENTRETENIMENTO
ESPORTES
SAÚDE
TECNOLOGIA
VIAGEM & GASTRONOMIA
Sobre a CNN Brasil16/10/2024, 15:28 Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições | CNN Brasil

page: 2
source: ./paris.pdf

conquistada pela delegação verde e amarela até hoje.
Nas edições de Paris 1924, Los Angeles 1932 e Berlim 1936 o Brasil não somou nenhuma medalha.
Ranking de medalhas de ouro do Brasil
Tóquio 2020 – 7 ouros
Rio 2016 – 7 ouros
Atenas 2004 – 5 ouros
Paris 2024 – 3 ouros
Londres 2012 – 3 ouros
Pequim 2008 – 3 ouros
Atlanta 1996 – 3 ouros
Moscou 1980 – 2 ouros
Barcelona 1992 – 2 ouros
Los Angeles 1984 – 1 ouro
Seul 1988 – 1 ouro
Antuérpia 1920 – 1 ouro
Helsinque 1952 – 1 ouro

page: 0
source: ./paris.pdf

Rebeca Andrade conquista o ouro no solo e é reverenciada por dupla dos Estados Unidos • Elsa/Getty Images
Compartilhar matéria
ouvir notícia
0:00 1.0x
Esportes Futebol Brasileirão Basquete Automobilismo Tênis eSports Apostas16/10/2024, 15:28 Brasil fecha Paris 2024 com 20 medalhas; veja comparação com outras edições | CNN Brasil
https://www.cnnbrasil.com.br/esportes/olimpiadas/resumo-brasil-paris-2024-medalhas-comparacao-outras-edicoes/ 1/9
--------------------
Você é um assistente para tarefas de perguntas e respostas.
Caso a "pergunta" não seja efetivamente uma pergunta, aja normalmente, como uma interação qualquer.
Caso contrário, você deve seguir as seguintes regras para responder às perguntas:
    - Use o contexto para responder à pergunta.
    - Pense em como chegar a resposta, responda e explique como chegou a ela.
    - Caso a pergunta não possa ser deduzida através do contexto, responda com "Não sei", caso contrário, responda com a resposta.
    - Preste bastante atenção no contexto e na pergunta para responder corretamente.
```

É possível customizar esse comportamento alterando os prompts manualmente.

Além de tracing, o LangFuse também oferece diversas funcionalidades, como evaluations, prompt versioning e outros, sendo *vital* para qualquer sistema LLM em produção.

## Material complementar

Apenas tocamos a superfície! Mostramos o RAG mais básico, também conhecido como "vanilla RAG" ou "baseline RAG".

Há **diversos** tipos de RAG:
- [Adaptative](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/)
- [Agentic](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/)
- [Corrective](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag/)
- [Self](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/)
- [Graph](https://microsoft.github.io/graphrag/)
- ...

Porém, com essa base vocês podem explorar e entender melhor esses outros tipos de RAG e porque eles foram criados e quais problemas eles resolvem.

### Links

Providers:
- [OpenAI](https://openai.com/)
- [Anthropic](https://www.anthropic.com/)
- [Sabiá](https://www.maritaca.ai/)
- [Amazon Bedrock](https://aws.amazon.com/bedrock/)
- [Azure ML](https://azure.microsoft.com/pt-br/products/machine-learning/generative-ai)
- [Google AI](https://ai.google.dev/)

Self-hosting:
- [Ollama](https://ollama.com/)
- [vLLM](https://docs.vllm.ai/en/latest/getting_started/quickstart.html)
- [llama.cpp](https://github.com/ggerganov/llama.cpp)

Frameworks:
- [LangChain](https://www.langchain.com/)
- [LlamaIndex](https://www.llamaindex.ai/)
- [CrewAI](https://www.crewai.com/)
- [AutoGen](https://microsoft.github.io/autogen)

Vector Databases:
- [Qdrant](https://qdrant.tech/)
- [Weaviate](https://weaviate.io/)
- [Chroma](https://www.trychroma.com/)
- [Milvus](https://milvus.io/)
- [Faiss](https://faiss.ai/)
- [Pinecone](https://www.pinecone.io/)
- [Astra](https://www.datastax.com/products/datastax-astra)

Prompt Engineering:
- [Prompting Guide](https://www.promptingguide.ai/techniques)
- [ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/courses/chatgpt-prompt-eng)

Observability:
- [LangFuse](https://langfuse.com/)
- [LangSmith](https://www.langchain.com/langsmith)
- [LiteralAI](https://literalai.com/)
- [Opik](https://www.comet.com/site/products/opik/)
- [OpenLLMetry](https://www.traceloop.com/openllmetry)
- [LlamaTrace/Phoenix](https://phoenix.arize.com/llamatrace/)

# OBRIGADO!